### This notebook is for sentiment data collecting using gnews

In [40]:
from gnews import GNews
import pandas as pd
import nltk
from rake_nltk import Rake
from tqdm import tqdm

In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/maissenbenjemaa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
data_sentiment = pd.read_csv('sentiment_gpt4_only.csv')

In [13]:
#extracting key words
def extract_keywords(sentence):
    rake_nltk_var = Rake()
    rake_nltk_var.extract_keywords_from_text(sentence)
    keyword_extracted = rake_nltk_var.get_ranked_phrases()
    return keyword_extracted

In [15]:
data_sentiment['headline'] = data_sentiment['Title'].apply(extract_keywords)

In [16]:
news = pd.DataFrame

,Date,Title,Sentiment,headline
0,2010-01-04 00:00:00,Global Stocks and Commodities Rally on First T...,positive,"[first trading day, global stocks, commodities..."
1,2010-01-04 00:00:00,Dollar Slumps Amid Worldwide Manufacturing Imp...,negative,[dollar slumps amid worldwide manufacturing im...
2,2010-01-04 00:00:00,Oil Prices Surge Above $81 a Barrel Due to U.S...,negative,"[oil prices surge, cold weather, barrel due, u..."
3,2010-01-04 00:00:00,"S&P 500 Sees 1.6 Percent Increase, Hits 15-Mon...",positive,"[p 500 sees 1, 6 percent increase, month high,..."
4,2010-01-04 00:00:00,"Argentina's Merval Index Reaches Record High, ...",positive,"[merval index reaches record high, bovespa sur..."
...,...,...,...,...
8134,2011-12-30 00:00:00,Earnings Season Begins with JPMorgan and Citi...,indecisive,"[earnings season begins, citigroup results exp..."
8135,2011-12-30 00:00:00,June Inflation Report Expected to Show Price ...,indecisive,"[june inflation report expected, show price gr..."
8136,2011-12-30 00:00:00,Chinese Trade Data Expected to Show Slower Ex...,negative,"[show slower export growth, chinese trade data..."
8137,2011-12-30 00:00:00,S&P 500 Exchange-Traded Fund Slips 0.5 Percen...,negative,"[traded fund slips 0, p 500 exchange, hours tr..."


In [35]:
google_news = GNews()
google_news.start_date = (2010, 1, 4) 
google_news.end_date = (2011, 12, 30)

In [38]:
dict = {}

In [44]:
#Searching for new titles using the titles in the dataset
for title in tqdm(data_sentiment['Title']):
    res = google_news.get_news(title)
    for resultat in res[:1]:
        pub_date = pd.to_datetime(resultat['published date'])
        dict [pub_date]= resultat['title']

100%|██████████| 8139/8139 [4:14:30<00:00,  1.88s/it]   


In [45]:
#generating new dataframe
dates = list(dict.keys())
titles = list(dict.values())
df = pd.DataFrame({'date': dates, 'title': titles}) 
df

,date,title
0,2010-02-04 08:00:00,"CNNMoney.com Market Report - Feb. 4, 2010 - CNN"
1,2011-08-08 07:00:00,Debt crisis sends financial markets into turmo...
2,2011-10-04 07:00:00,"Market Report - Oct. 4, 2011 - CNNMoney - CNN"
3,2024-06-04 20:47:00,Hunter Biden's wife lashes out at former Trump...
4,2024-06-04 17:37:30,Hunter Biden's wife lashes out at ex-Trump adv...
...,...,...
895,2024-06-03 07:29:00,India stocks hit record highs to lead gains in...
896,2024-05-31 04:16:40,US Stocks Dip As Tech Shares Lead Decline - Fi...
897,2024-02-15 08:00:00,"Stock Market News: Dow Jumps 348 Points, Erasi..."
898,2024-06-05 20:11:52,"Stock market today: S&P 500, Nasdaq hit record..."


In [49]:
df['date'] = pd.to_datetime(df['date'],utc=True)

In [53]:
indexes_to_drop = df[df['date'].dt.year >= 2012].index

In [54]:
df.drop(indexes_to_drop, inplace=True)

In [ ]:
#saving the dataframe
df.to_csv('sent_data_base.csv',index=False)